#### Import standard Python modules

In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

#### Import Azure ML SDK modules

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#### Check Azure ML SDK version

In [3]:
print(azureml.core.VERSION)

1.42.0


#### Create Azure ML Workspace

In [5]:
AZ_SUBSCRIPTION_ID=' '
ws = Workspace.create(name='salary',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='ameer-diaz-321',
                      create_resource_group=True,
                      location='southeastasia'
                     )

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


Deploying KeyVault with name salarykeyvaultcc6f1e5608.
Deploying AppInsights with name salaryinsightsf4c248535c.
Deployed AppInsights with name salaryinsightsf4c248535c. Took 9.87 seconds.
Deploying StorageAccount with name salarystoragee8ff62d845f.
Deployed KeyVault with name salarykeyvaultcc6f1e5608. Took 27.57 seconds.
Deployed StorageAccount with name salarystoragee8ff62d845f. Took 35.85 seconds.
Deploying Workspace with name salary.
Deployed Workspace with name salary. Took 25.06 seconds.


#### Write configuration to local file

In [6]:
ws.write_config()

#### Create Azure ML Experiment

In [7]:
exp = Experiment(workspace=ws, name='salexp')

#### Start logging metrics

In [8]:
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

#### Load salary dataset

In [9]:
sal = pd.read_csv('data/sal.csv',header=0, index_col=None)
X = sal[['x']]
y = sal['y']

#### Split the train and test data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#### Train the model

In [11]:
lm = LinearRegression() 
lm.fit(X_train,y_train) 

LinearRegression()

#### Freeze the model

In [12]:
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

['outputs/sal_model.pkl']

#### Test the model

In [14]:
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename)
y=loaded_model.predict([[2]])[0]
print(y)

106797.22222222222


#### Log metrics to Azure ML Experiment

In [15]:
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [16]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [17]:
print(run.get_portal_url())

https://ml.azure.com/runs/e34ae860-8807-40a2-9622-4e3b3941dbe8?wsid=/subscriptions/b7584e96-fb23-4dc1-94ce-dd5c4d572e8f/resourcegroups/mi2/workspaces/salary&tid=58a02e86-b56a-4884-a632-a2f325b9e5e7


#### Register the model

In [18]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


#### Define Azure ML Deploymemt configuration

In [19]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

#### Create enviroment configuration file

In [20]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.8.12

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



#### Create Azure ML Scoring file

In [21]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


#### Deploy the model to Azure Container Instance


In [22]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")

Wall time: 3.22 ms


C:\Users\Ameer\anaconda3\envs\aml\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  This is separate from the ipykernel package so we can avoid doing imports until


#### Expose web service

In [23]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='salary-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

C:\Users\Ameer\anaconda3\envs\aml\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  """


Creating image
Running.........................................................
Succeeded
Image creation operation finished for image salary-svc:1, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-18 01:39:10+05:30 Generating deployment configuration.
2022-06-18 01:39:10+05:30 Submitting deployment to compute.
2022-06-18 01:39:17+05:30 Checking the status of deployment salary-svc..
2022-06-18 01:40:54+05:30 Checking the status of inference endpoint salary-svc.
Succeeded
ACI service creation operation finished, operation "Succeeded"


#### Get the Web Service URL

In [24]:
print(service.scoring_uri)

http://78c15216-e00c-4add-841f-82054385ca63.southeastasia.azurecontainer.io/score


In [47]:
import requests

headers = {
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
}

json_data = {
    'data': [
        [
            2,
        ],
    ],
}

response = requests.post('http://78c15216-e00c-4add-841f-82054385ca63.southeastasia.azurecontainer.io/score', headers=headers, json=json_data)

print(response.text)

"[106797.22222222222]"


#### Delete Workspace and clean up resources

In [31]:
ws.delete()